In [4]:
import numpy as np
import pandas as pd
import os
import requests
from time import sleep
from pathlib import Path

import openai
from transformers import GPT2Tokenizer

In [5]:
local_dir = Path(os.getcwd())
data_dir = local_dir / 'llm-detect-ai-generated-text'
external_data_dir = local_dir / 'external-data'

### Load Data

In [6]:
train_essays = pd.read_csv(data_dir / 'train_essays.csv')
train_prompts = pd.read_csv(data_dir / 'train_prompts.csv')

train_essays = train_essays.merge(train_prompts, on='prompt_id', how='left')

In [7]:
# Persuade data
persuade_data = pd.read_csv(external_data_dir / 'Persuade' / 'persuade_2.csv')
persuade_data.rename(columns={'prompt_name': 'prompt_id', 'holistic_essay_score': 'score'}, inplace=True)
persuade_data['generated'] = 0
persuade_data['dataset'] = 'persuade'
persuade_data['source'] = persuade_data['prompt_id']

## Tokenize Text and Determine Distriution of Word Counts

In [8]:
train_essays['introduction'] = [essay.split('\n')[0] for essay in train_essays['text']]
train_essays['conclusion'] = [essay.split('\n')[-1] for essay in train_essays['text']]

In [9]:
# Tokenize text
model_architecture = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_architecture)

def tokenize_text(text):
    return tokenizer.encode(text, return_tensors='pt')

text_token_lengths = []
for text in train_essays['text']:
    tokens = tokenize_text(text)
    length = len(tokens[0])
    text_token_lengths.append(length)
    
train_essays['text_token_length'] = text_token_lengths

# Tokenize introduction
introduction_token_lengths = []
for text in train_essays['introduction']:
    tokens = tokenize_text(text)
    length = len(tokens[0])
    introduction_token_lengths.append(length)
    
train_essays['introduction_token_length'] = introduction_token_lengths

# Tokenize conclusion
conclusion_token_lengths = []
for text in train_essays['conclusion']:
    tokens = tokenize_text(text)
    length = len(tokens[0])
    conclusion_token_lengths.append(length)
    
train_essays['conclusion_token_length'] = conclusion_token_lengths

# Calculate introduction length ratio
train_essays['introduction_length_ratio'] = train_essays['introduction_token_length'] / train_essays['text_token_length']
valid_introduction_condition = (train_essays['introduction_length_ratio'] >= 0.05) & (train_essays['introduction_length_ratio'] <= 0.25)
train_essays['valid_introduction'] = np.where(valid_introduction_condition, 1, 0)

# Calculate conclusion length ratio
train_essays['conclusion_length_ratio'] = train_essays['conclusion_token_length'] / train_essays['text_token_length']
valid_conclusion_condition = (train_essays['conclusion_length_ratio'] >= 0.05) & (train_essays['conclusion_length_ratio'] <= 0.25)
train_essays['valid_conclusion'] = np.where(valid_conclusion_condition, 1, 0)

Token indices sequence length is longer than the specified maximum sequence length for this model (1081 > 1024). Running this sequence through the model will result in indexing errors


In [10]:
# Calculate essay length distribution
def calculate_essay_len_distribution(token_lengths, bins):
    # Calculate histogram bin counts with the specified number of bins
    counts, bin_edges = np.histogram(token_lengths, bins=bins)
    probabilities = counts / counts.sum()
    
    return (bin_edges, probabilities)

def sample_from_essay_len_distribution(length_distribution):
    bin_edges, probabilities = length_distribution
    # Choose a bin index, then sample uniformly from within that bin
    bin_index = np.random.choice(len(probabilities), p=probabilities)

    return np.random.uniform(bin_edges[bin_index], bin_edges[bin_index + 1])

essay_length_distribution = calculate_essay_len_distribution(train_essays['text_token_length'], bins=50)

## Generate AI Essay Based on Train Data

In [28]:
def generate_ai_essay(
    ai_role=None, instruction=None, source_text=None, 
    essay_introduction=None, essay_conclusion=None,
    final_instruction=None, length_distribution=None, 
    model='gpt-3.5-turbo'
):
    """
    Generates an essay response from the OpenAI ChatGPT API.

    This function initializes a conversation with the OpenAI API using the provided arguments,
    which define the role of the AI, the instructions, the source text, and any final instructions
    for generating the essay. It then sends this conversation to the API and returns the generated
    essay response.

    Parameters
    ----------
    ai_role : str, optional
        The role description of the AI in the conversation, which instructs the AI on how to behave
        in the context of the essay (default is None, which should be set to a proper description).
    instruction : str, optional
        The initial set of instructions for the essay that the AI is to write (default is None).
    source_text : str, optional
        The source text information that the AI can use to form its response (default is None).
    essay_introduction : str, optional
        The start of the essay that the AI is to write (default is None).
    essay_conclusion : str, optional
        The end of the essay that the AI is to write (default is None).
    model : str, optional
        The identifier for the OpenAI language model to be used (default is 'gpt-3.5-turbo').
    final_instruction : str, optional
        Any final instructions to be considered by the AI when generating the essay (default is None).
    length_distribution : tuple, optional
        A tuple containing the the distribution of essay lengths to be generated by the AI (default is None).

    Returns
    -------
    str
        The AI-generated essay response.
    """
    
    # Initialize the conversation with the AI
    if ai_role is None:
        formatted_ai_role = """Role:
            
        You are a student writing an academic argumentative essay for a class assignment."""
        
    else:
        formatted_ai_role = f"""Role:
            
        {ai_role}"""
        
    conversation = [
        {'role': 'system', 'content': formatted_ai_role}
    ]
    
    # Set instructions if provided
    if instruction is None:
        formatted_instructions = """Instructions:
        
        Use the source text to write the essay."""
    
    else:
        formatted_instructions = f"""Instructions: 
            
        {instruction}"""
        conversation.append({'role': 'user', 'content': formatted_instructions})
    
    # Set source text if provided
    if source_text is not None:
        formatted_source_text = f"""Source Text:
            
        {source_text}"""
        conversation.append({'role': 'user', 'content': formatted_source_text})
        
    # Set the essay introduction if provided
    if essay_introduction is not None:
        formatted_essay_introduction = f"""Essay Introduction:
        
        {essay_introduction}"""
        conversation.append({'role': 'user', 'content': formatted_essay_introduction})
        
    # Set the essay conclusion if provided
    if essay_conclusion is not None:
        formatted_essay_conclusion = f"""Essay Conclusion:
        
        {essay_conclusion}"""
        conversation.append({'role': 'user', 'content': formatted_essay_conclusion})
    
    # Set final instructions if provided
    if final_instruction is not None:
        if length_distribution is not None:
            essay_length = sample_from_essay_len_distribution(length_distribution) - 100
            essay_length = round(essay_length / 50) * 50
        else:
            essay_length = 500
        
        formatted_final_instruction = f"""Final Instructions:
        
        {final_instruction}. The essay should be {essay_length} words long."""
        
        # Adjust final instructions if introduction and/or conclusion are provided
        if essay_introduction is not None and essay_conclusion is not None:
            formatted_final_instruction = 'Using the Essay Introduction: as your introduction paragraph and the Essay Conclusion: as your conclusion paragraph, p' + final_instruction[1:]
        elif essay_introduction is not None:
            formatted_final_instruction = 'Using the Essay Introduction: as your introduction paragraph, p' + final_instruction[1:]
        elif essay_conclusion is not None:
            formatted_final_instruction = 'Using the Essay Conclusion: as your conclusion paragraph, p' + final_instruction[1:]
        
        conversation.append({'role': 'user', 'content': formatted_final_instruction})
    
    # Send the messages to the model
    openai_client = openai.OpenAI(api_key=Path('/Users/claygendron/Git/Files/OpenAI.txt').read_text())
    try:
        response = openai_client.chat.completions.create(
            model=model,
            messages=conversation
        )
        
        essay = response.choices[0].message.content
    
    except requests.exceptions.ReadTimeout as e:
        print('The request to OpenAI timed out. You may want to retry.')
        essay = None
        
    except openai.OpenAIError as e:
        print(f'An OpenAI error occurred: {e}')
        essay = None
        
    except Exception as e:
        print(f'An unexpected error occurred: {e}')
        essay = None
        
    return essay

### AI Roles

In [29]:
ai_role_1 = """You are a student who is required to write an argumentative essay for a class assignment.
The essay should have a clear, concise, and defined thesis statement in the first paragraph.
Use logical transitions between the introduction, body, and conclusion.
Include body paragraphs that provide evidential support, considering different points of view and explaining how the evidence supports the thesis.
The conclusion should readdress the thesis in light of the evidence provided without introducing new information.

You will be given a set of instructions and source text for the assignment.
Be sure to address the questions being asked in the instructions and use information in the source text to build your essay."""

ai_role_2 = """Assume the role of a university-level student tasked with crafting a persuasive argumentative essay.
Your opening paragraph must feature a distinct thesis statement that encapsulates your argument.
Ensure a fluid transition between your introduction, the supporting arguments, and your conclusion.
Develop body paragraphs that bolster your thesis with substantive evidence and address a variety of perspectives.
The concluding section should not introduce new facts but should reconsider the thesis in the context of the evidence discussed.

Specific instructions and a source text will guide your composition.
It is imperative that you engage directly with the questions posited in the instructions and integrate insights from the source text into your argument."""

ai_role_3 = """You are acting as a high school student who must formulate a compelling argumentative essay as part of your coursework.
Begin with a strong thesis statement early in your essay to guide your narrative.
Transition seamlessly from your introduction, through the main points in your body paragraphs, to a coherent conclusion.
In the body of your essay, incorporate concrete evidence to substantiate your thesis and thoughtfully consider opposing viewpoints.
Finish with a conclusion that revisits your thesis statement, now enlightened by the evidence provided, without adding new information.

You will be prompted with specific instructions and a textual resource for reference.
Your response should thoroughly tackle the proposed questions and judiciously use the provided source text to enhance your argument."""

ai_role_4 = """Envision yourself as a debate team member in a collegiate setting, preparing a written argumentative essay for competition.
Your essay must start with a precise and potent thesis statement.
Narrative coherence is key; maintain clear and logical transitions from the introduction, to the body, and through to the conclusion.
Support each claim in your body paragraphs with robust evidence while also addressing counterarguments.
The essay should conclude by revisiting the thesis in the light of the evidence, without venturing into uncharted territory.

Instructions and a source text will be provided to anchor your essay.
It is crucial that your essay confronts the posed questions head-on and effectively utilizes the source text as a foundation for your argument."""

ai_role_5 = """Picture yourself as a scholar engaging in academic debate through the written word, composing an argumentative essay.
Initiate your discourse with a definitive thesis statement in the opening paragraph.
Ensure that each section of your essay - the introduction, the body, and the conclusion - is interconnected by smooth and rational transitions.
The body of your essay should be founded on evidence that not only underpins your thesis but also acknowledges alternative perspectives.
Your conclusion should echo the thesis, illuminated by the evidence discussed, while abstaining from presenting new data.

You will be equipped with instructions and a source text to inform your essay.
It is essential for your essay to directly answer the instructions' queries and to construct your argument upon the premises provided in the source text."""

ai_roles = [ai_role_1, ai_role_2, ai_role_3, ai_role_4, ai_role_5]

In [30]:
revised_ai_role_1 = """You are a student who is required to write an argumentative essay for a class assignment. 
Think of this as not just an assignment, but as a chance to express your own views. 
Start with a clear thesis statement in your first paragraph - think about what you really want to say. 
As you move from the introduction to the body and then to the conclusion, make sure your ideas flow logically, like you're having a conversation with the reader. 
In your body paragraphs, provide evidence and different viewpoints - imagine you're explaining this to a friend, why do you believe what you believe? 

In your conclusion, come back to your thesis, reflect on it, maybe even question it, based on the evidence but avoid throwing in completely new ideas at the last minute. 
You'll have instructions and a source text to guide you, but don't just stick to these - ask yourself, what do these questions really mean? 
How does the source text speak to your own experiences or knowledge? Make this essay uniquely yours. 
Also, use contractions and informal language, like you're talking to a close colleague. Relaxed but formal."""

revised_ai_role_2 = """Envision yourself as a university-level student crafting a persuasive essay. 
Your opening should present a thesis that reflects your unique viewpoint. 
Imagine explaining your argument to a fellow student, ensuring smooth transitions between your introduction, body, and conclusion. 
Use the body paragraphs to reinforce your thesis with real-world evidence, as if you're engaging in a lively classroom discussion. 

Wrap up by revisiting your thesis, enriched by the journey of your essay, but steer clear of new revelations. 
Dive into the given instructions and source text, but intertwine them with your own experiences and thoughts for a genuinely compelling narrative.
Also, use contractions and informal language, like you're talking to a close colleague. Relaxed but formal."""

revised_ai_role_3 = """Picture yourself as a high school student diving into an argumentative essay. 
Lead with a thesis statement that truly speaks to you, early in your essay. 
As you weave through the introduction, body, and conclusion, think of it as telling a story that gradually unfolds. 
In the body, when you bring in evidence, imagine how you'd debate this topic with classmates, considering various angles. 

Conclude by reflecting on your thesis, informed by all the evidence, but avoid throwing in new twists at the end. 
While you'll have specific instructions and a text for reference, don't forget to inject your personal flair and interpretations into the mix.
Also, use contractions and informal language, like you're talking to a close colleague. Relaxed but formal."""

revised_ai_role_4 = """Think of yourself as a debate team member in college, preparing an argumentative essay. 
Kick off with a thesis statement that you're passionate about. 
As you transition from the introduction to the body and then to the conclusion, imagine guiding your audience through your argument step by step. 
Support your points in the body with strong evidence, as if you're in a debate, addressing and countering opposing views. 

Conclude by tying everything back to your thesis, enlightened by the evidence, but without veering off into new ideas. 
Use the provided instructions and source text as a foundation, but also let your own voice and reasoning shine through.
Also, use contractions and informal language, like you're talking to a close colleague. Relaxed but formal."""

revised_ai_role_5 = """Visualize yourself as a scholar engaged in an academic debate through an essay. 
Your opening paragraph should echo your distinct viewpoint with a definitive thesis statement. 
As you connect the introduction, body, and conclusion, think of it as weaving a tapestry of ideas. 
In the body, back your thesis with evidence, but also ponder other perspectives, like you're in a round-table discussion. 

End by revisiting your thesis, now enlightened, but without adding unexpected new data. 
While you will follow specific instructions and a source text, let your personal insights and style infuse the essay, making it uniquely yours.
Also, use contractions and informal language, like you're talking to a close colleague. Relaxed but formal."""

ai_roles = [revised_ai_role_1, revised_ai_role_2, revised_ai_role_3, revised_ai_role_4, revised_ai_role_5]

### Final Instructions

In [18]:
final_instruction_1 = """Please write your essay.
Write in paragraph format without the use of a title or headers."""

final_instruction_2 = """Please write your academic paper.
Write in paragraph format without the use of a title or headers."""

final_instruction_3 = """Please write your response.
Write in paragraph format without the use of a title or headers."""

final_instructions = [final_instruction_1, final_instruction_2, final_instruction_3]

### Essay Introductions and Conclusions

In [19]:
# Introductions
prompt_0_conditions = (train_essays['prompt_id'] == 0) & (train_essays['valid_introduction'] == 1) & (train_essays['generated'] == 0)
prompt_0_introductions = train_essays[prompt_0_conditions]['introduction'].values

prompt_1_conditions = (train_essays['prompt_id'] == 1) & (train_essays['valid_introduction'] == 1) & (train_essays['generated'] == 0)
prompt_1_introductions = train_essays[prompt_1_conditions]['introduction'].values

# Conclusions
prompt_0_conditions = (train_essays['prompt_id'] == 0) & (train_essays['valid_conclusion'] == 1) & (train_essays['generated'] == 0)
prompt_0_conclusions = train_essays[prompt_0_conditions]['conclusion'].values

prompt_1_conditions = (train_essays['prompt_id'] == 1) & (train_essays['valid_conclusion'] == 1) & (train_essays['generated'] == 0)
prompt_1_conclusions = train_essays[prompt_1_conditions]['conclusion'].values

### Generate AI Essays

In [20]:
ai_essays = []
for model in ['gpt-4-1106-preview']:
    num_essays = 100
    prompt_list = [i % 2 for i in range(num_essays)]
    for i in range(num_essays):
        prompt = prompt_list[i]
        
        ai_role = np.random.choice(ai_roles)
        instruction = train_prompts['instructions'][prompt]
        source_text = train_prompts['source_text'][prompt]
        essay_introduction = np.random.choice(prompt_0_introductions) if prompt == 0 else np.random.choice(prompt_1_introductions)
        essay_conclusion = np.random.choice(prompt_0_conclusions) if prompt == 0 else np.random.choice(prompt_1_conclusions)
        final_instruction = np.random.choice(final_instructions)
        length_distribution = essay_length_distribution
        
        # Randomly make some parameters None
        ai_rand = np.random.rand()
        if ai_rand < 0.20:
            ai_role = None
            
        instruction_source_rand = np.random.rand()
        if instruction_source_rand < 0.50:
            pass
        
        else:
            if instruction_source_rand < 0.75:
                instruction = None
            else:
                source_text = None
        
        essay_introduction_rand = np.random.rand()
        if essay_introduction_rand < 0.50:
            essay_introduction = None
            
        essay_conclusion_rand = np.random.rand()
        if essay_conclusion_rand < 0.50:
            essay_conclusion = None
            
        # Generate the essay
        ai_essay = None
        counter = 0
        while ai_essay is None and counter < 5:
            counter += 1
            ai_essay = generate_ai_essay(
                ai_role=ai_role, instruction=instruction, source_text=source_text, 
                essay_introduction=essay_introduction, essay_conclusion=essay_conclusion,
                final_instruction=final_instruction, length_distribution=length_distribution, 
                model=model
            )
            
            if model == 'gpt-4-1106-preview':
                sleep(15)
        
        # Create the essay row
        id = model + '_' + str(prompt) + '_' + str(i)
        data = [id, prompt, ai_essay, 1, model, ai_role, instruction, source_text, essay_introduction, essay_conclusion, final_instruction]
        ai_essays.append(data)
        
        print(f'Generated essay {i} for prompt {prompt} with model {model}')

Generated essay 0 for prompt 0 with model gpt-4-1106-preview
Generated essay 1 for prompt 1 with model gpt-4-1106-preview
Generated essay 2 for prompt 0 with model gpt-4-1106-preview
Generated essay 3 for prompt 1 with model gpt-4-1106-preview
Generated essay 4 for prompt 0 with model gpt-4-1106-preview
Generated essay 5 for prompt 1 with model gpt-4-1106-preview
Generated essay 6 for prompt 0 with model gpt-4-1106-preview
Generated essay 7 for prompt 1 with model gpt-4-1106-preview
Generated essay 8 for prompt 0 with model gpt-4-1106-preview
Generated essay 9 for prompt 1 with model gpt-4-1106-preview
Generated essay 10 for prompt 0 with model gpt-4-1106-preview
Generated essay 11 for prompt 1 with model gpt-4-1106-preview
Generated essay 12 for prompt 0 with model gpt-4-1106-preview
Generated essay 13 for prompt 1 with model gpt-4-1106-preview
Generated essay 14 for prompt 0 with model gpt-4-1106-preview
Generated essay 15 for prompt 1 with model gpt-4-1106-preview
Generated essay 16

In [21]:
train_ai_essays_cols = ['id', 'prompt_id', 'text', 'generated', 'model', 'ai_role', 'instruction', 'source_text', 'essay_introduction', 'essay_conclusion', 'final_instruction']
train_ai_essays = pd.DataFrame(ai_essays, columns=train_ai_essays_cols)

In [ ]:
train_ai_essays.to_csv(data_dir / 'train_essays_ai_gpt4-preview-revised.csv', index=False)

## Generate AI Essay Based on Persuade Data

In [15]:
def generate_ai_essay(
    ai_role=None, data=None,
    model='gpt-3.5-turbo'
):
    """
    Generates an essay response from the OpenAI ChatGPT API.

    This function initializes a conversation with the OpenAI API using the provided arguments,
    which define the role of the AI, the instructions, the source text, and any final instructions
    for generating the essay. It then sends this conversation to the API and returns the generated
    essay response.

    Parameters
    ----------
    ai_role : str, optional
        The role description of the AI in the conversation, which instructs the AI on how to behave
        in the context of the essay (default is None, which should be set to a proper description).
    data : dataframe, optional
        A dataframe containing the data to be used a source text for the AI (default is None).

    Returns
    -------
    str
        The AI-generated essay response.
    """
    
    # Initialize the conversation with the AI
    if ai_role is None:
        ai_role = """You are a student writing an academic argumentative essay for a class assignment."""
        
    conversation = [
        {'role': 'system', 'content': ai_role}
    ]
    
    # Set source text for the AI
    if data is None:
        formatted_instructions = """Source Text:
        
        No source text. Choose a topic of substance and write an essay about it."""
        group_prompt = None
        group_score = None
    else:
        source_instructions = f"""Source Text:
            
        Please write a synthesized essay based on the following instructions:

        1. Analyze the next three essays you receive for their style, tone, skill level, precision, and formatting.
        2. Inherit the distinctive elements from these essays, including their style of argumentation, tone, level of detail, and overall writing skill.
        3. Pay close attention to any recurring errors, such as spelling mistakes and grammatical errors, and incorporate similar errors into your essay. Adopt the vocabulary and level of language complexity used in the source essays, including any technical or subject-specific terms, as well as overall essay length.
        4. Ensure that the synthesized essay reflects a blend of the source materials, maintaining coherence and logical flow while integrating the key characteristics of each source essay. If the sources contrast each other, you can choose how to reflect this contrast in your essay.
        5. The essay that is created should be a synthesis of the source essays, not a copy of them, a reflection of them."""
        conversation.append({'role': 'user', 'content': source_instructions})
        
        # Randomly select 3 essays of the same prompt and score
        prompt_and_score = pd.DataFrame(data.groupby(['prompt_id', 'score']).size().reset_index(name='count'))
        prompt_and_score = prompt_and_score[prompt_and_score['count'] >= 20]
        group = prompt_and_score.sample(n=1)
        group_prompt = group['prompt_id'].values[0]
        group_score = group['score'].values[0]

        group_data = data[(data['prompt_id'] == group_prompt) & (persuade_data['score'] == group_score)]
        sample = group_data.sample(n=3)
        
        for i in range(len(sample)):
            essay = sample['text'].values[i]
            formatted_source_text = f"""Source Text{i + 1}:
            
            {essay}"""
            conversation.append({'role': 'user', 'content': formatted_source_text})
        
        final_instruction = """Final Instructions:
        
        Please make your response only the synthesized essay. Do not include the source text or mention of it."""
        conversation.append({'role': 'user', 'content': formatted_source_text})
    
    # Send the messages to the model
    openai_client = openai.OpenAI(api_key=Path('/Users/claygendron/Git/Files/OpenAI.txt').read_text())
    try:
        response = openai_client.chat.completions.create(
            model=model,
            messages=conversation
        )
        
        essay = response.choices[0].message.content
    
    except requests.exceptions.ReadTimeout as e:
        print('The request to OpenAI timed out. You may want to retry.')
        essay = None
        
    except openai.OpenAIError as e:
        print(f'An OpenAI error occurred: {e}')
        essay = None
        
    except Exception as e:
        print(f'An unexpected error occurred: {e}')
        essay = None
        
    return essay, group_prompt, group_score

### Generate AI Essays

In [22]:
ai_essays = []
for model in ['gpt-4-1106-preview']:
    num_essays = 250
    for i in range(num_essays):
        # Generate the essay
        ai_essay = None
        counter = 0
        while ai_essay is None and counter < 5:
            counter += 1
            ai_essay, ai_prompt, ai_score = generate_ai_essay(data=persuade_data, model=model)
            
            if model == 'gpt-4-1106-preview':
                sleep(15)
        
        # Create the essay row
        id = model + '_' + str(ai_prompt) + '_' + str(ai_score) + '_' + str(i)
        data = [id, ai_prompt, ai_score, ai_essay, 1, model]
        ai_essays.append(data)
        
        print(f'Generated essay {i} for prompt {ai_prompt} with model {model}')

Generated essay 0 for prompt Distance learning with model gpt-4-1106-preview
Generated essay 1 for prompt Mandatory extracurricular activities with model gpt-4-1106-preview
Generated essay 2 for prompt Grades for extracurricular activities with model gpt-4-1106-preview
Generated essay 3 for prompt "A Cowboy Who Rode the Waves" with model gpt-4-1106-preview
Generated essay 4 for prompt Seeking multiple opinions with model gpt-4-1106-preview
Generated essay 5 for prompt Driverless cars with model gpt-4-1106-preview
Generated essay 6 for prompt The Face on Mars with model gpt-4-1106-preview
Generated essay 7 for prompt Cell phones at school with model gpt-4-1106-preview
Generated essay 8 for prompt "A Cowboy Who Rode the Waves" with model gpt-4-1106-preview
Generated essay 9 for prompt The Face on Mars with model gpt-4-1106-preview
Generated essay 10 for prompt "A Cowboy Who Rode the Waves" with model gpt-4-1106-preview
Generated essay 11 for prompt Summer projects with model gpt-4-1106-p

In [23]:
train_ai_essays_cols = ['id', 'prompt_id', 'score_id', 'text', 'generated', 'model']
train_ai_essays = pd.DataFrame(ai_essays, columns=train_ai_essays_cols)

In [24]:
train_ai_essays.to_csv(data_dir / 'train_essays_ai_persuade_1.csv', index=False)